<a href="https://colab.research.google.com/github/tylerlum/ufc_automated_scoring_system/blob/main/UFC_Automated_Scoring_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UFC Automated Scoring System

The goal of this notebook is to:
* Explore the FightMetrics webpage to scrape the fight and fighter information we need
* Store the fight and fighter data into csv files
* Preprocess the data
* Train and evaluate a neural network to predict fight outcomes

Still functional as of April 2021

## Get information about all fighters

In [ ]:
import pandas as pd
from tqdm import tqdm
import numpy as np

In [ ]:
def get_all_fighters():
  '''Get pandas table of all UFC fighters (Name, Height, Weight, Reach, Record, etc.)'''
  from string import ascii_lowercase

  all_fighters_tables = []
  for c in tqdm(ascii_lowercase):
    all_fighters_url = f"http://ufcstats.com/statistics/fighters?char={c}&page=all"
    all_fighters_table = pd.read_html(all_fighters_url)[0]
    all_fighters_tables.append(all_fighters_table)

  all_fighters = pd.concat(all_fighters_tables)
  return all_fighters

In [ ]:
ALL_FIGHTERS = get_all_fighters()
ALL_FIGHTERS.head()

100%|██████████| 26/26 [00:16<00:00,  1.62it/s]


,First,Last,Nickname,Ht.,Wt.,Reach,Stance,W,L,D,Belt
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Tom,Aaron,NaN,--,155 lbs.,--,NaN,5.0,3.0,0.0,NaN
2,Danny,Abbadi,The Assassin,"5' 11""",155 lbs.,--,Orthodox,4.0,6.0,0.0,NaN
3,David,Abbott,Tank,"6' 0""",265 lbs.,--,Switch,10.0,15.0,0.0,NaN
4,Shamil,Abdurakhimov,Abrek,"6' 3""",235 lbs.,"76.0""",Orthodox,20.0,5.0,0.0,NaN


In [ ]:
ALL_FIGHTERS.dtypes

First        object
Last         object
Nickname     object
Ht.          object
Wt.          object
Reach        object
Stance       object
W           float64
L           float64
D           float64
Belt        float64
dtype: object

## Clean fighter data

TODO: Convert height, weight, reach to floats.

In [ ]:
ALL_FIGHTERS = ALL_FIGHTERS.replace('--', np.NaN)
ALL_FIGHTERS.head()

,First,Last,Nickname,Ht.,Wt.,Reach,Stance,W,L,D,Belt
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Tom,Aaron,NaN,NaN,155 lbs.,NaN,NaN,5.0,3.0,0.0,NaN
2,Danny,Abbadi,The Assassin,"5' 11""",155 lbs.,NaN,Orthodox,4.0,6.0,0.0,NaN
3,David,Abbott,Tank,"6' 0""",265 lbs.,NaN,Switch,10.0,15.0,0.0,NaN
4,Shamil,Abdurakhimov,Abrek,"6' 3""",235 lbs.,"76.0""",Orthodox,20.0,5.0,0.0,NaN


## Helper functions

In [ ]:
def get_fighters(fighters_string, all_fighter_tables):
    '''Parses string containing two fighter names. Uses all_fighter_tables to remove ambiguity in parsing. Returns each fighter name'''
    for i, row in all_fighter_tables.iterrows():
        fighter_name = f'{row["First"]} {row["Last"]}'
        if fighters_string.startswith(fighter_name):
            first_fighter = fighter_name
            second_fighter = fighters_string[len(fighter_name)+1:]
            break
    return first_fighter, second_fighter

## Get a list of all UFC events

In [ ]:
from urllib.request import urlopen
from string import ascii_uppercase
from dateutil import parser
from datetime import datetime

In [ ]:
ALL_PAST_EVENTS_URL = "http://ufcstats.com/statistics/events/completed?page=all"

In [ ]:
def get_all_events(all_past_events_url):
    '''Takes in URL to all past events. Returns list of urls, each one representing a UFC event'''
    def find_latest_index_with_char(string, chars):
        for i in reversed(range(len(string))):
            if string[i] in chars:
                return i
        return -1

    # Get all event names
    all_past_events_tables = pd.read_html(all_past_events_url)[0]
    all_past_events_tables = all_past_events_tables[all_past_events_tables["Name/date"].notna()]

    event_names = []
    for i, row in all_past_events_tables.iterrows():
        date_index = find_latest_index_with_char(row["Name/date"], ascii_uppercase)
        event_name = row["Name/date"][:date_index-1].strip()
        date = parser.parse(row["Name/date"][date_index:])
        if date < datetime.now():
            event_names.append(event_name)

    # Hacky way to get urls for all events
    all_past_events_html = urlopen(all_past_events_url).read().decode("utf-8")
    all_urls = []
    for event_name in event_names:
        new_substring = all_past_events_html[:all_past_events_html.index(event_name)]
        url_index = new_substring.rfind("http://ufcstats.com/event-details/")
        url_string = new_substring[url_index:]
        url_string = url_string[:url_string.find("\"")]
        all_urls.append(url_string)
    return all_urls

In [ ]:
# Events
ALL_EVENT_URLS = get_all_events(ALL_PAST_EVENTS_URL)
print(f"Got {len(ALL_EVENT_URLS)} events")
print(ALL_EVENT_URLS)

Got 558 events
['http://ufcstats.com/event-details/4a35913bd9aa4161', 'http://ufcstats.com/event-details/9114c8ded5ccd71d', 'http://ufcstats.com/event-details/6597b611f1c32555', 'http://ufcstats.com/event-details/c36e1f4fa755ffb4', 'http://ufcstats.com/event-details/8c90c1563972e44d', 'http://ufcstats.com/event-details/6e2b1d631832921d', 'http://ufcstats.com/event-details/81b57acd6975ac06', 'http://ufcstats.com/event-details/bfe95ec546692b13', 'http://ufcstats.com/event-details/3f7c14c7eca7195d', 'http://ufcstats.com/event-details/a1153013cb5f628f', 'http://ufcstats.com/event-details/4304992c2acc187b', 'http://ufcstats.com/event-details/56116537d71a578c', 'http://ufcstats.com/event-details/307064d3e0f036c2', 'http://ufcstats.com/event-details/e49c2db95e572dc8', 'http://ufcstats.com/event-details/d0d43cb9b14f231c', 'http://ufcstats.com/event-details/398b440e73d55b8b', 'http://ufcstats.com/event-details/992c82450d96f726', 'http://ufcstats.com/event-details/bbb15f301e4a490a', 'http://ufcs

## Get a list of UFC fights

In [ ]:
def get_all_fights_in_event(past_event_url, get_results=False):
    '''Takes in a single URL to a past event. Returns list, with each element representing a UFC fight.
       If get_results=True, each element is a tuple of a fight url and winner.
       If get_results=False, each element is a fight url'''
    past_event_html = urlopen(past_event_url).read().decode("utf-8")
    past_event_tables = pd.read_html(past_event_url)[0]  # Will be length 1 list

    # Read through the raw event html. Find the URL and the winner of each fight.
    body = past_event_html[past_event_html.index("<body "):]
    fight_urls, winners = [], []
    for i, row in past_event_tables.iterrows():
        result = row["W/L"].split(' ')[0]
        before_result = body[:body.index(result)]
        begin = before_result[before_result.rfind("http://ufcstats.com/fight-details/"):]
        url = begin[:begin.find("\"")]
        if result == "win":
            winner, loser = get_fighters(row["Fighter"], ALL_FIGHTERS)
        else:
            winner = None

        # Store url and winner
        fight_urls.append(url)
        winners.append(winner)

        # Move text forward
        body = body[body.index(result)+len(result):]

    if get_results:
        return fight_urls, winners
    else:
        return fight_urls

In [ ]:
def get_all_fights(all_event_urls, num_events=None):
    '''Takes in list of URLs to past events. Returns list of urls, each representing a UFC fight.
       Set num_events to be the number of events to get fights from. Set to None if want all.'''
    if num_events is None:
        num_events = len(all_event_urls)
    
    all_fight_urls, all_winners = [], []
    for i, event_url in enumerate(tqdm(all_event_urls[:num_events])):
        # Cap the number of events
        if i == num_events:
            break

        # For each event, get the fight urls and winners
        fight_urls, winners = get_all_fights_in_event(event_url, get_results=True)
        all_fight_urls.extend(fight_urls)
        all_winners.extend(winners)
    return all_fight_urls, all_winners

In [ ]:
FIGHT_URLS, WINNERS = get_all_fights(ALL_EVENT_URLS, num_events=10)
print(f"Got {len(FIGHT_URLS)} fights")
print(FIGHT_URLS)
print(WINNERS)




  0%|          | 0/10 [00:00<?, ?it/s]


 10%|█         | 1/10 [00:01<00:16,  1.78s/it]


 20%|██        | 2/10 [00:04<00:15,  1.97s/it]


 30%|███       | 3/10 [00:06<00:13,  1.95s/it]


 40%|████      | 4/10 [00:07<00:11,  1.92s/it]


 50%|█████     | 5/10 [00:10<00:09,  1.99s/it]


 60%|██████    | 6/10 [00:12<00:08,  2.09s/it]


 70%|███████   | 7/10 [00:13<00:05,  1.83s/it]


 80%|████████  | 8/10 [00:15<00:03,  1.91s/it]


 90%|█████████ | 9/10 [00:17<00:01,  1.97s/it]


100%|██████████| 10/10 [00:20<00:00,  2.06s/it]

Got 114 fights
['http://ufcstats.com/fight-details/f67aa0b16e16a9ea', 'http://ufcstats.com/fight-details/f2c82f027201ba21', 'http://ufcstats.com/fight-details/a850ba762f3c5334', 'http://ufcstats.com/fight-details/0b2c3c4306620d6d', 'http://ufcstats.com/fight-details/a33ecd2c88154890', 'http://ufcstats.com/fight-details/fdfe4e7cb2d0b665', 'http://ufcstats.com/fight-details/25d4376e00b16510', 'http://ufcstats.com/fight-details/627c0a50cff30405', 'http://ufcstats.com/fight-details/8d634c9240f161dd', 'http://ufcstats.com/fight-details/24ff385ca78c80d3', 'http://ufcstats.com/fight-details/ddd63237f1b857a6', 'http://ufcstats.com/fight-details/71b81d8199a0fa19', 'http://ufcstats.com/fight-details/646eeca9447c6965', 'http://ufcstats.com/fight-details/0992795fd0fa06d0', 'http://ufcstats.com/fight-details/ac9a8b73e58f3d71', 'http://ufcstats.com/fight-details/e27c28928ed4390f', 'http://ufcstats.com/fight-details/998296216e7171ed', 'http://ufcstats.com/fight-details/7aebf01b5afc0109', 'http://ufcs

## Get fight tables

TODO: Add exception handling. Sometimes gets weird Document Empty errors. Investigate why.

In [ ]:
def get_labeled_fight_tables(fight_url):
    '''Convert fight url to dictionary of pandas tables of information.
       Before gave a list of tables that was hard to understand.
       Now have Totals, Per Round Totals, Significant Strikes, Per Round Significant Strikes
       May return None if has issues'''
    try:
        fight_tables = pd.read_html(fight_url)
    except Exception:
        print(f"Failure on fight_url = {fight_url}")
        return None
    
    labeled_fight_tables = {}
    labeled_fight_tables['Totals'] = fight_tables[0]
    labeled_fight_tables['Per Round Totals'] = fight_tables[1]
    labeled_fight_tables['Significant Strikes'] = fight_tables[2]
    labeled_fight_tables['Per Round Significant Strikes'] = fight_tables[3]
    return labeled_fight_tables

In [ ]:
RAW_FIGHT_TABLES_LIST = []
for url in tqdm(FIGHT_URLS):
    RAW_FIGHT_TABLES_LIST.append(get_labeled_fight_tables(url))
RAW_FIGHT_TABLES_LIST[0]['Totals'].head()





  0%|          | 0/114 [00:00<?, ?it/s]



  1%|          | 1/114 [00:00<00:58,  1.95it/s]



  2%|▏         | 2/114 [00:01<01:05,  1.70it/s]



  3%|▎         | 3/114 [00:02<01:19,  1.40it/s]



  4%|▎         | 4/114 [00:03<01:19,  1.38it/s]



  4%|▍         | 5/114 [00:03<01:16,  1.43it/s]



  5%|▌         | 6/114 [00:04<01:16,  1.40it/s]



  6%|▌         | 7/114 [00:05<01:13,  1.45it/s]



  7%|▋         | 8/114 [00:05<01:12,  1.46it/s]



  8%|▊         | 9/114 [00:06<01:10,  1.49it/s]



  9%|▉         | 10/114 [00:07<01:09,  1.51it/s]



 10%|▉         | 11/114 [00:07<01:09,  1.48it/s]



 11%|█         | 12/114 [00:08<01:08,  1.49it/s]



 11%|█▏        | 13/114 [00:08<01:04,  1.55it/s]



 12%|█▏        | 14/114 [00:09<01:04,  1.55it/s]



 13%|█▎        | 15/114 [00:10<01:03,  1.55it/s]



 14%|█▍        | 16/114 [00:10<01:05,  1.50it/s]



 15%|█▍        | 17/114 [00:11<01:01,  1.57it/s]



 16%|█▌        | 18/114 [00:12<01:01,  1.56it/s]



 17%|█▋        | 19/114 [0

Failure on fight_url = 






 39%|███▉      | 45/114 [00:28<00:38,  1.77it/s]



 40%|████      | 46/114 [00:29<00:40,  1.68it/s]



 41%|████      | 47/114 [00:29<00:41,  1.63it/s]



 42%|████▏     | 48/114 [00:30<00:40,  1.63it/s]

Failure on fight_url = 






 44%|████▍     | 50/114 [00:31<00:33,  1.89it/s]



 45%|████▍     | 51/114 [00:31<00:34,  1.84it/s]



 46%|████▌     | 52/114 [00:32<00:33,  1.83it/s]



 46%|████▋     | 53/114 [00:33<00:35,  1.71it/s]



 47%|████▋     | 54/114 [00:33<00:36,  1.63it/s]



 48%|████▊     | 55/114 [00:34<00:36,  1.62it/s]



 49%|████▉     | 56/114 [00:34<00:34,  1.67it/s]



 50%|█████     | 57/114 [00:35<00:35,  1.62it/s]



 51%|█████     | 58/114 [00:36<00:34,  1.64it/s]



 52%|█████▏    | 59/114 [00:36<00:34,  1.59it/s]



 53%|█████▎    | 60/114 [00:37<00:34,  1.58it/s]



 54%|█████▎    | 61/114 [00:38<00:33,  1.56it/s]



 54%|█████▍    | 62/114 [00:38<00:33,  1.55it/s]



 55%|█████▌    | 63/114 [00:39<00:32,  1.55it/s]



 56%|█████▌    | 64/114 [00:40<00:31,  1.57it/s]



 57%|█████▋    | 65/114 [00:40<00:30,  1.59it/s]



 58%|█████▊    | 66/114 [00:41<00:32,  1.50it/s]



 59%|█████▉    | 67/114 [00:42<00:30,  1.55it/s]



 60%|█████▉    | 68/114 [00:42<00:29,  1.58it/s]



 61%|███

,Fighter,KD,Sig. str.,Sig. str. %,Total str.,Td,Td %,Sub. att,Rev.,Ctrl
0,Robert Whittaker Kelvin Gastelum,0 0,150 of 284 62 of 209,52% 29%,169 of 303 70 of 218,4 of 7 1 of 5,57% 20%,0 0,0 0,4:19 0:26


## Clean fight information

Separate each fighter's information into a different column

TODO: Lots of stuff to improve. Don't just use Totals table. Use round by round and significant strikes. Can also use non integer information, total attempted strikes (not just landed), control time, etc. All of those being ignored right now.

In [ ]:
def parse_string(row_string):
    '''Break string into two parts: one for fighter 1 and one for fighter 2
       Eg. 150 of 284  62 of 209 => (150 of 284, 62 of 209)'''
    string_split = row_string.split(" ")
    first_fighter_stat = " ".join(string_split[:len(string_split)//2])
    second_fighter_stat = " ".join(string_split[len(string_split)//2+1:])
    return first_fighter_stat, second_fighter_stat

In [ ]:
def convert_to_int_or_double_if_possible(string):
    '''Convert string to int or double if possible
       If has a percent sign, tries to remove it and continue.'''
    def isfloat(value):
        try:
            float(value)
            return True
        except ValueError:
            return False

    if "%" in string:
        string = string.strip("%")

    if isfloat(string) and float(string).is_integer():
        return int(string)
    if isfloat(string):
        return float(string)
    return string

In [ ]:
def process_fight(raw_fight_tables):
    '''Takes in a raw pandas fight table. Returns a pandas dataframe representing the fight statistics'''
    # Focus on totals table
    totals_table = raw_fight_tables['Totals']

    # Change column from "Fighter" to "Name" to make things cleaner later
    totals_table = totals_table.rename(columns={'Fighter': 'Name'})
    
    # Break up columns.
    # Eg. "Name" => "Fighter 1 Name", "Fighter 2 Name"
    # "KD" => "Fighter 1 KD", "Fighter 2 KD"
    new_columns = []
    for column in totals_table.columns:
        new_columns.append(f"Fighter 1 {column}")
        new_columns.append(f"Fighter 2 {column}")

    # Go through each row and break up the data into the columns
    new_rows = []
    for i, row in totals_table.iterrows():
        new_row = []
        for column in totals_table.columns:
            # Split string at the center space
            stat1, stat2 = parse_string(row[column])
            # print(stat1)
            # Remove things after the first space
            # TODO: Update this to capture more information
            # Like sig strikes 17 of 37, the 37 is lost
            stat1Space = stat1.index(' ') if ' ' in stat1 else -1
            stat2Space = stat2.index(' ') if ' ' in stat2 else -1
            if stat1Space != -1:
                stat1 = stat1[:stat1Space]
            if stat2Space != -1:
                stat2 = stat2[:stat2Space]
            
            # Convert string to float or int if possible
            stat1 = convert_to_int_or_double_if_possible(stat1)
            stat2 = convert_to_int_or_double_if_possible(stat2)

            # Add to row
            new_row.append(stat1)
            new_row.append(stat2)

        new_rows.append(new_row)

    # Bring together into new dataframe, then only store the numerical values
    # TODO: Process better to keep more info, not throw so much away
    df = pd.DataFrame(new_rows, columns=new_columns)
    df = df.select_dtypes([np.number])

    # Add in names, using smarter parsing
    fighters_string = totals_table["Name"][0]  # Only 1 row table
    fighter1, fighter2 = get_fighters(fighters_string, ALL_FIGHTERS)
    df['Fighter 1 Name'] = fighter1
    df['Fighter 2 Name'] = fighter2
    return df

In [ ]:
def process_raw_fight_tables(raw_fight_tables, winner):
    '''Takes in a raw fight table and the name of the fight winner. Returns a cleaned pandas table.'''
    fight_table = process_fight(raw_fight_tables)
    if fight_table["Fighter 1 Name"][0] == winner:
        label = 1
    elif fight_table["Fighter 2 Name"][0] == winner:
        label = 2
    else:
        print(f'ERROR: fight_table["Fighter 1 Name"]={fight_table["Fighter 1 Name"]}, fight_table["Fighter 2 Name"]={fight_table["Fighter 2 Name"]}, winner={winner}')
        label = 0
    fight_table['Winner'] = label
    return fight_table

In [ ]:
FIGHT_TABLE = []
for raw_fight_table, winner in tqdm(zip(RAW_FIGHT_TABLES_LIST, WINNERS), total=len(RAW_FIGHT_TABLES_LIST)):
    # Handle invalid readings
    if raw_fight_table is not None:
        FIGHT_TABLE.append(process_raw_fight_tables(raw_fight_table, winner)) 
FIGHT_TABLE = pd.concat(FIGHT_TABLE, ignore_index=True)







  0%|          | 0/114 [00:00<?, ?it/s]





  1%|          | 1/114 [00:00<00:36,  3.10it/s]





  3%|▎         | 3/114 [00:00<00:30,  3.67it/s]





  4%|▍         | 5/114 [00:00<00:25,  4.25it/s]





  5%|▌         | 6/114 [00:01<00:26,  4.05it/s]





  6%|▌         | 7/114 [00:01<00:26,  4.05it/s]





  7%|▋         | 8/114 [00:01<00:21,  4.86it/s]





  8%|▊         | 9/114 [00:01<00:20,  5.21it/s]





  9%|▉         | 10/114 [00:01<00:17,  5.80it/s]





 10%|▉         | 11/114 [00:02<00:22,  4.60it/s]





 12%|█▏        | 14/114 [00:02<00:17,  5.65it/s]





 13%|█▎        | 15/114 [00:02<00:20,  4.93it/s]





 14%|█▍        | 16/114 [00:02<00:19,  4.91it/s]





 15%|█▍        | 17/114 [00:03<00:17,  5.41it/s]





 16%|█▌        | 18/114 [00:03<00:17,  5.36it/s]





 18%|█▊        | 20/114 [00:03<00:13,  6.84it/s]





 18%|█▊        | 21/114 [00:03<00:17,  5.29it/s]





 19%|█▉        | 22/114 [00:03<00:16,  5.57it/s]





 20%|██        | 23/114 [00:03<00:15, 

ERROR: fight_table["Fighter 1 Name"]=0    Joseph Benavidez
Name: Fighter 1 Name, dtype: object, fight_table["Fighter 2 Name"]=0    Askar Askarov
Name: Fighter 2 Name, dtype: object, winner=Askar Askar








 59%|█████▉    | 67/114 [00:11<00:09,  5.10it/s]





 61%|██████    | 69/114 [00:12<00:08,  5.36it/s]





 61%|██████▏   | 70/114 [00:12<00:08,  5.14it/s]





 63%|██████▎   | 72/114 [00:12<00:07,  5.54it/s]





 64%|██████▍   | 73/114 [00:12<00:07,  5.48it/s]

ERROR: fight_table["Fighter 1 Name"]=0    Montana De La Rosa
Name: Fighter 1 Name, dtype: object, fight_table["Fighter 2 Name"]=0    Mayra Bueno Silva
Name: Fighter 2 Name, dtype: object, winner=None








 66%|██████▌   | 75/114 [00:13<00:06,  5.81it/s]





 68%|██████▊   | 77/114 [00:13<00:05,  6.68it/s]





 69%|██████▉   | 79/114 [00:13<00:04,  8.01it/s]





 70%|███████   | 80/114 [00:13<00:04,  7.57it/s]





 72%|███████▏  | 82/114 [00:13<00:04,  6.88it/s]





 73%|███████▎  | 83/114 [00:14<00:05,  5.53it/s]





 74%|███████▎  | 84/114 [00:14<00:05,  5.13it/s]





 75%|███████▍  | 85/114 [00:14<00:05,  5.72it/s]





 76%|███████▋  | 87/114 [00:14<00:03,  7.02it/s]





 77%|███████▋  | 88/114 [00:14<00:05,  4.99it/s]





 78%|███████▊  | 89/114 [00:15<00:04,  5.09it/s]





 80%|███████▉  | 91/114 [00:15<00:04,  4.97it/s]





 81%|████████  | 92/114 [00:15<00:05,  4.27it/s]





 82%|████████▏ | 93/114 [00:16<00:05,  3.78it/s]





 83%|████████▎ | 95/114 [00:16<00:03,  4.79it/s]





 84%|████████▍ | 96/114 [00:16<00:04,  4.33it/s]





 85%|████████▌ | 97/114 [00:16<00:03,  4.26it/s]





 86%|████████▌ | 98/114 [00:17<00:04,  3.76it/s]





 87%|████████▋ | 99/11

In [ ]:
FIGHT_TABLE.head()

,Fighter 1 KD,Fighter 2 KD,Fighter 1 Sig. str.,Fighter 2 Sig. str.,Fighter 1 Sig. str. %,Fighter 2 Sig. str. %,Fighter 1 Total str.,Fighter 2 Total str.,Fighter 1 Td,Fighter 2 Td,Fighter 1 Td %,Fighter 2 Td %,Fighter 1 Sub. att,Fighter 2 Sub. att,Fighter 1 Rev.,Fighter 2 Rev.,Fighter 1 Name,Fighter 2 Name,Winner
0,0,0,150,62,52,29,169,70,4,1,57.0,20.0,0,0,0,0,Robert Whittaker,Kelvin Gastelum,1
1,0,0,105,88,47,45,109,99,0,0,NaN,NaN,0,0,0,0,Andrei Arlovski,Chase Sherman,1
2,0,0,21,27,44,49,32,91,0,8,NaN,33.0,0,2,0,0,Abdul Razak Alhassan,Jacob Malkoun,2
3,0,1,66,55,58,46,84,72,3,0,60.0,0.0,1,0,0,0,Tracy Cortez,Justine Kish,1
4,0,0,90,80,46,47,90,82,0,4,NaN,44.0,0,0,0,0,Luis Pena,Alexander Munoz,1


In [ ]:
FIGHT_TABLE.tail()

,Fighter 1 KD,Fighter 2 KD,Fighter 1 Sig. str.,Fighter 2 Sig. str.,Fighter 1 Sig. str. %,Fighter 2 Sig. str. %,Fighter 1 Total str.,Fighter 2 Total str.,Fighter 1 Td,Fighter 2 Td,Fighter 1 Td %,Fighter 2 Td %,Fighter 1 Sub. att,Fighter 2 Sub. att,Fighter 1 Rev.,Fighter 2 Rev.,Fighter 1 Name,Fighter 2 Name,Winner
107,0,0,57,38,67,64,135,70,4,1,66.0,33.0,0,0,0,1,Karol Rosa,Joselyne Edwards,1
108,0,0,37,36,78,62,71,85,0,7,NaN,77.0,1,1,0,0,Molly McCann,Lara Procopio,2
109,0,0,41,23,37,48,73,46,2,3,66.0,33.0,0,1,0,1,SeungWoo Choi,Youssef Zalal,1
110,0,0,55,8,72,28,115,24,4,0,66.0,NaN,0,0,0,1,Timur Valiev,Martin Day,1
111,1,0,5,2,83,66,5,2,0,0,NaN,NaN,0,0,0,0,Ode Osbourne,Jerome Rivera,1
